In [36]:
import pandas as pd
from Bio import SeqIO
from Utils.Clustering.Cluster_Containments_Utils import *
from Utils.Reference_Guided_Scaffolding.Scaffold_Using_References_Utils import *

import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib.patches import Rectangle
rcParams = {'font.size': 20, 
            'font.weight': 'normal', 
            'font.family': 'sans-serif',
            'axes.unicode_minus':False, 
            'axes.labelweight':'normal', 
            'xtick.labelsize' : 16,
            'ytick.labelsize' : 16,
            'axes.labelsize': 20,
            'axes.spines.right' : False,
            'axes.spines.top' : False,
            'axes.spines.left' : False, 
           }

plt.rcParams.update(rcParams)

osa_len = 2932766
osb_len = 3046682

colors = {'C' : 'red', 'L':'blue', 'P':'olive', '-':'grey', 'M':'orange', 'I':'gold', 
          'S':'purple', 'H':'tan', 'J':'yellow', 'E':'cyan', 'G':'magenta', 'V':'lime',
          'O':'teal', 'F':'black', 'Q':'yellow', 'T':'lightcoral', 'KL':'brown', 'K':'peachpuff', 'FP':'plum', 
          'MU':'peru', 'KLT':'sienna', 'KT':'maroon', 'NU':'indigo', 'U':'darkolivegreen',
          'GM':'crimson', 'NOU':'thistle', 'NPTU':'dodgerblue', 'EGP':'firebrick', 'IQ':'mediumslateblue', 
          'PT':'lightsalmon', 'D':'khaki', 'OU':'darkseagreen', 'ET':'lawngreen', 'CO':'sandybrown',
          'CH':'tan', 'UW':'royalblue', 'HP':'goldenrod', 'EQ':'springgreen', 'HJ':'cadetblue', 'CDZ':'pink', 
          'FG':'slategrey'}

def gbff_to_dataframe(gbff_file):
    records = SeqIO.parse(gbff_file, "genbank")

    data = []
    for record in records:
        for feature in record.features:
            if feature.type == "CDS":
                row = {
                    "locus_tag": feature.qualifiers.get("locus_tag", [""])[0],
                    "old_locus_tag": feature.qualifiers.get("old_locus_tag", [""])[0],
                    "gene": feature.qualifiers.get("gene", [""])[0],
                    "product": feature.qualifiers.get("product", [""])[0],
                    "Start": feature.location.start,
                    "End": feature.location.end,
                    "Orientation": feature.location.strand,
                }
                if row["Orientation"] == 1:
                    row["Orientation"] == "+"
                else:
                    row["Orientation"] == "-"
                data.append(row)

    return pd.DataFrame(data)

def Load_Prodigal_GBFF(filepath):
    lines = open(filepath).readlines()
    op = []
    ctr = 1
    for i in range(len(lines)):
        l = lines[i]
        l = l.replace("\n","")
        l = l.strip()
        if l == "//":
            ctr = 1
        else:
            if l.startswith("DEFINITION"):
                l = l.replace("DEFINITION ","")
                splits = l.split(";")
                for s in splits:
                    if s.startswith("seqhdr"):
                        sequence_id = s.replace("seqhdr=","").replace("\"","")
                    if s.startswith("seqlen"):
                        seqlen = int(s.replace("seqlen=","").replace("\"",""))
            if l.startswith("CDS"):
                l = l.replace("CDS","").strip()
                if l.startswith("complement"):
                    orientation = '-'
                else:
                    orientation = '+'
                l = l.replace("complement","").replace("(","").replace(")","").replace("<","").replace(">","")
                start,end = l.split("..")
                i += 1
                record = lines[i]
                record = record.strip()
                if record.startswith("/note"):
                    record = record.replace("/note=","").replace("\"","")
                    splits = record.split(";")
                    d = {'Query':sequence_id, 'Pred':sequence_id+'_'+str(ctr), 'Qlen':seqlen, 
                         'Orientation':orientation, 'Start':start, 'End':end}
                    for s in splits[:-1]:
                        s = s.replace("\n","")
                        key, value = s.split('=')
                        d[key] = value
                    ctr += 1
                    op.append(d)
    df_Prodigal_Hits = pd.DataFrame(op)
    df_Prodigal_Hits['Start'] = df_Prodigal_Hits['Start'].astype(int)
    df_Prodigal_Hits['End'] = df_Prodigal_Hits['End'].astype(int)
    return df_Prodigal_Hits[['Query','Pred','Orientation','Start','End','Qlen','ID','partial','conf']]

def Make_Counts(df_osa, df_osb, osa_len, osb_len):
    osa_indicator = {}
    osb_indicator = {}
    osa_gbl_counts = np.zeros(osa_len)
    osb_gbl_counts = np.zeros(osb_len)
    try: osa_samples = set(df_osa['Sample'].tolist())
    except KeyError: osa_samples = set({})
    try: osb_samples = set(df_osb['Sample'].tolist())
    except KeyError: osb_samples = set({})
        
    for g in osa_samples:
        try:
            osa = np.zeros(osa_len)
            temp_osa = df_osa[df_osa['Sample'] == g]
            starts, ends, lengths = temp_osa['Start'].tolist(), temp_osa['End'].tolist(), temp_osa['Length'].tolist()
            
            for i in range(len(starts)):
                start, end = int(min(starts[i],ends[i])), int(max(starts[i],ends[i]))
                if abs(start-end) == lengths[i]:    
                    osa[start:end] += 1
                    osa_gbl_counts[start:end] += 1
                else:
                    osa[end:osa_len] += 1
                    osa[0:start] += 1
                    osa_gbl_counts[end:osa_len] += 1
                    osa_gbl_counts[0:start] += 1
            osa[np.isnan(osa)] = 0
            osa_indicator[g] = osa
        except KeyError: osa_indicator[g] = osa
            
    for g in osb_samples:    
        try:
            osb = np.zeros(osb_len)
            temp_osb = df_osb[df_osb['Sample'] == g].reset_index()
            starts, ends, lengths = temp_osb['Start'].tolist(), temp_osb['End'].tolist(), temp_osb['Length'].tolist()
            
            for i in range(len(starts)):
                start, end = int(min(starts[i],ends[i])), int(max(starts[i],ends[i]))
                if abs(start-end) == lengths[i]:    
                    osb[start:end] += 1
                    osb_gbl_counts[start:end] += 1
                else:
                    osb[end:osa_len] += 1
                    osb[0:start] += 1
                    osb_gbl_counts[end:osb_len] += 1
                    osb_gbl_counts[0:start] += 1
            osb[np.isnan(osb)] = 0
            osb_indicator[g] = osb
        except KeyError: osb_indicator[g] = osb
    return osa_indicator, osb_indicator, osa_gbl_counts, osb_gbl_counts

def Make_Circle_Plots(ax, values, color, text, ylim, lw = 5):
    angles = np.linspace(0, 2*np.pi, len(values), endpoint=False)
    ax.plot(angles, values, linewidth=lw, color = color)
    ax.grid(False)
    ax.set_ylim([0, ylim])
    ax.set_yticklabels([])
    ax.set_xticks([])
    ax.text(0,0, text, horizontalalignment='center', verticalalignment='center',size = 20)
    ax.spines['polar'].set_visible(False)

def Plot_Gene_Annotations(df, ax, width, idx, start, end, offset=2500, color = 'gray'):
    filt_start = min(start, end)-offset
    filt_end = filt_start+offset+width+offset
    df_filt = df.loc[(df['Start'] >= filt_start) & (df['End'] <= filt_end)].copy()
    df_filt['Start'] = df_filt['Start'] - min(start, end)
    df_filt['End'] = df_filt['End'] - min(start, end)
    df_gene = df_filt[df_filt["gene"]!=""].copy()
    df_gene['annotation'] = df_gene["gene"]
    df_empty = df_filt[df_filt["gene"]==""].copy()
    df_empty['annotation'] = df_empty["product"]
    df_flanking = pd.concat([df_gene, df_empty], axis = 0)

    ax.plot([-1*offset, width+offset],[idx,idx], linewidth = 12, color = color, alpha = 0.6)
    starts = df_flanking['Start'].tolist()
    ends =  df_flanking['End'].tolist()
    orientations = df_flanking['Orientation'].tolist()
    annotation = df_flanking['annotation'].tolist()
    #if "seed_ortholog" in 
    for i in range(0, len(starts)):
        if orientations[i] == 1:
            ax.arrow(starts[i],idx-1,ends[i]-starts[i]+1,0, width = 0.15, 
                     length_includes_head = True, head_length = 150)
        else:
            ax.arrow(ends[i],idx-1,starts[i]-ends[i]+1,0, width = 0.15, 
                    length_includes_head = True, head_length = 150)
        loc = starts[i]
        if " " not in annotation[i]: loc = starts[i]+(ends[i]-starts[i])/4
        ax.text(loc, idx-1+0.25, annotation[i], size = 12, rotation = 20)
    idx = idx - 2
    return ax, idx

In [2]:
gbff_path = '/Users/harihara/Downloads/GCF_000013205.1_ASM1320v1_genomic.gbff'
df_osa_gbff = gbff_to_dataframe(gbff_path)

gbff_path = '/Users/harihara/Downloads/GCF_000013225.1_ASM1322v1_genomic.gbff'
df_osb_gbff = gbff_to_dataframe(gbff_path)
T = pd.concat([df_osa_gbff, df_osb_gbff], axis = 0)

In [3]:
grp_path = '/Users/harihara/Mount/Hotsprings_Variant_Structure_Data_Analysis/Synechococcus/containment_clusters.txt'
df_novel_filtered = pd.read_csv(grp_path, sep = "\t")
d = df_novel_filtered.groupby('GroupID')['Contig'].apply(list).to_dict()
d_representatives = df_novel_filtered.groupby('GroupID')['RepresentativeContig'].apply(list).to_dict()

In [4]:
novel_contigs = {}
novel_contig_path = '/Users/harihara/Mount-2/hotspring_metagenome/Synechococcus_paper_analysis/\
Hotsprings_Variant_Structure/'

samples = listdir(novel_contig_path+'OSA/')
for s in samples:
    if s.startswith("Hot"):
        print(s)
        df_osa = pd.read_csv(f"{novel_contig_path}OSA/{s}/reference_guided_scaffolds/Ref_Guided_Scaffolds.OSA.txt", 
                             sep = "\t")
        df_osa.loc[(df_osa['Start'] < 0), 'Start'] += osa_len
        df_osa.loc[(df_osa['End'] < 0), 'End'] += osa_len
        df_osa_grp = df_osa.sort_values(by = ['Contig','Start']).groupby(['Contig']).apply(Max_Clique_Interval_Graph)
    
        df_osb = pd.read_csv(f"{novel_contig_path}OSB/{s}/reference_guided_scaffolds/Ref_Guided_Scaffolds.OSB.txt", 
                             sep = "\t")
        df_osb.loc[(df_osb['Start'] < 0), 'Start'] += osb_len
        df_osb.loc[(df_osb['End'] < 0), 'End'] += osb_len
        df_osb_grp = df_osb.sort_values(by = ['Contig','Start']).groupby(['Contig']).apply(Max_Clique_Interval_Graph)
        novel_contigs[s.replace(".txt","")] = {'OSA':df_osa_grp,'OSB':df_osb_grp}

HotsprottomLayer_FD
HotsprSampOS1260_FD
HotsprSampleMS65_FD
Hotspr20Samplem2_FD
HotsprSampleOSM1_FD
HotsprSampleMS60_FD
HotsprSampleOSM3_FD
HotsprSampleMSe1_FD
HotsprSampleOSM2_FD
HotsprSampleMS55_FD
HotsprSampleOS65_FD
Hotspr20Samplet1_FD
HotsprSampOS1265_FD
Hotspr20SampleT9_FD
HotsprSampleOS55_FD
Hotspr2Sampleee2_FD
Hotspr2Sample149_FD
HotsprSampleOS50_FD
Hotspr2Sample148_FD
HotsprOSTMatCore_FD
HotsprottomLayer_2_FD
HotsprSampleOSM4_FD
HotsprSampleMS13_FD
HotsprSampleMSe4_FD
Hotspr20SampleP4_FD
HotsprSampleOS60_FD
HotsprSampleMS50_FD
HotsprSampleMSe2_FD
Hotspr20SampleT8_FD
Hotspr2SamplePe2_FD
HotsprSampleMSe3_FD
HotsprSampleR4cd_FD
Hotspr2Sampleme2_FD
HotsprSamplt10cd_FD


In [43]:
prodigal_out = '/Users/harihara/Mount/Hotsprings_Variant_Structure_Data_Analysis/Synechococcus/Prodigal/\
Representatives_Prodigal.out'
df_prodigal_hits = Load_Prodigal_GBFF(prodigal_out)
df_prodigal_hits = df_prodigal_hits.set_index('Pred')

eggnog_path = '/Users/harihara/Mount/Hotsprings_Variant_Structure_Data_Analysis/Synechococcus/EggNOG/\
Representatives.eggnog.out.emapper.annotations'
df_eggnog = pd.read_csv(eggnog_path, sep = "\t")
df_eggnog['RepresentativeContig'] = df_eggnog['#query'].apply(get_Query_ID)
df_eggnog = df_eggnog.rename(columns = {'#query':'Query'})
df_eggnog = df_eggnog.set_index('Query')
df_eggnog['COG_category'] = df_eggnog['COG_category'].fillna("-")
df_eggnog['Description'] = df_eggnog['Description'].fillna("Unspecified")
df_eggnog[['seed_0','seed_1']] = df_eggnog['seed_ortholog'].str.rsplit(".", n=1, expand = True)
df_prodigal_hits = df_prodigal_hits.join(df_eggnog)
df_prodigal_hits = df_prodigal_hits.reset_index()
df_prodigal_hits = df_prodigal_hits.merge(df_novel_filtered[['RepresentativeContig','GroupID']].drop_duplicates(),
                                          left_on = 'Query', right_on='RepresentativeContig', how = 'left')
df_prodigal_hits = pd.merge(df_prodigal_hits, T[['old_locus_tag','product','gene']], 
                            left_on = 'seed_1', right_on = 'old_locus_tag', how = "outer")
df_prodigal_hits['gene'] = df_prodigal_hits['gene'].fillna(df_prodigal_hits['Preferred_name'])
df_prodigal_hits['gene'] = df_prodigal_hits['gene'].fillna("")
df_prodigal_hits['product'] = df_prodigal_hits['product'].fillna("")


In [11]:
synechococcus_blast = '/Users/harihara/Mount/Hotsprings_Variant_Structure_Data_Analysis/Synechococcus/\
Representatives.Synechococcus.blast'

df_blast = pd.read_csv(synechococcus_blast, sep = "\t",
                       names=['qseqid', 'sseqid', 'pident', 'length', 'mismatch', 'gapopen', 'qlen', 
                              'qstart', 'qend', 'slen', 'sstart', 'send', 'evalue', 'bitscore'])
df_blast = df_blast.merge(df_novel_filtered[['RepresentativeContig','GroupID']].drop_duplicates(),
                          left_on = 'qseqid', right_on='RepresentativeContig', how = 'left')
df_blast['sseqid'] = df_blast['sseqid'].replace("gi|86604733|ref|NC_007775.1|","OSA")
df_blast['sseqid'] = df_blast['sseqid'].replace("gi|86607503|ref|NC_007776.1|","OSB")

grouped = df_blast.rename(columns = {'sseqid':'Genome'}).groupby(['qseqid','Genome'])
df_blast_grouped = pd.DataFrame()
df_blast_grouped['QCov'] = grouped.apply(Compute_Query_Coverage)
df_blast_grouped = df_blast_grouped.reset_index().pivot(index = ['qseqid'], columns=['Genome'], values=['QCov'])
df_blast_grouped = df_blast_grouped.fillna(0)
df_blast_grouped.columns = df_blast_grouped.columns.droplevel()
df_blast_grouped = df_blast_grouped[(df_blast_grouped['OSA'] < 75) & (df_blast_grouped['OSB'] < 75)]
representative_short_listed = df_blast_grouped.index.tolist()

['Hotspr20SampleP4_FD_OSA_k141_10411'] ['OSA'] 729 17.55829903978052
['Hotspr20SampleP4_FD_OSA_k141_10411'] ['OSB'] 729 99.86282578875172
['HotsprSampleMS13_FD_OSB_k141_51304'] ['OSB'] 729 99.86282578875172


In [9]:
out_dir = '/Users/harihara/Research-Activities/Plots/Hot_Spring_Plots/Synechococcus-Paper-New-Plots/'
if not isdir(out_dir):
    mkdir(out_dir)
if not isdir(out_dir+'Novel_Groups_11102024/'):
    mkdir(out_dir+'Novel_Groups_11102024/')

In [44]:
plt.rcParams.update(rcParams)
OSA_All_Counts, OSB_All_Counts = {}, {}
filtered_grp = df_novel_filtered[df_novel_filtered['RepresentativeContig'].isin(representative_short_listed)]

for g in list(filtered_grp['GroupID'].unique()):    
    print(g)
    contigs = d[g]+[d_representatives[g][0]]
    osa_contig_count, osb_contig_count = set({}), set({})
    osa_contigs, osb_contigs = [], []
    df_osa, df_osb = pd.DataFrame(), pd.DataFrame()
    
    for c in contigs:
        splits = c.split('_')
        contig = splits[-2]+'_'+splits[-1]
        genome = splits[-3].upper()
        sample = "_".join(splits[:-3])
        
        if genome == "OSA": osa_contig_count.add(sample)
        if genome == "OSB": osb_contig_count.add(sample)
        
        try:
            row = novel_contigs[sample][genome].loc[contig]
            row['Group'] = g
            row['Sample'] = sample
            if genome == 'OSA': df_osa = df_osa.append(row)
            elif genome == 'OSB': df_osb = df_osb.append(row)
        except:
            print("Missing...",sample,genome,contig)
            pass
    
    if len(df_osa) > 0: df_osa = df_osa.reset_index()
    if len(df_osb) > 0: df_osb = df_osb.reset_index()
    
    osa_counts, osb_counts, osa_gbl_counts, osb_gbl_counts = Make_Counts(df_osa, df_osb, osa_len, osb_len)
    osa_gbl_counts += 30
    osb_gbl_counts += 30
    for o in osa_counts:
        try: OSA_All_Counts[o] += osa_counts[o]
        except KeyError: OSA_All_Counts[o] = osa_counts[o]
    
    for o in osb_counts:
        try: OSB_All_Counts[o] += osb_counts[o]
        except KeyError: OSB_All_Counts[o] = osb_counts[o]
            
    fig = plt.figure(figsize=(16, 12))
    gs = GridSpec(nrows=2, ncols=2, height_ratios=[2.25, 3])
    ax0 = fig.add_subplot(gs[0, 0], projection = 'polar')
    ax1 = fig.add_subplot(gs[0, 1], projection = 'polar')
    ax2 = fig.add_subplot(gs[1, :])
    
#     osa_alignments_starts = df_blast.loc[(df_blast['GroupID'] == g) & (df_blast['sseqid'] == 'OSA') ,
#                                          'qstart'].tolist()
#     osa_alignments_ends = df_blast.loc[(df_blast['GroupID'] == g) & (df_blast['sseqid'] == 'OSA') ,
#                                          'qend'].tolist()
    
#     osb_alignments_starts = df_blast.loc[(df_blast['GroupID'] == g) & (df_blast['sseqid'] == 'OSB') ,
#                                          'qstart'].tolist()
#     osb_alignments_ends = df_blast.loc[(df_blast['GroupID'] == g) & (df_blast['sseqid'] == 'OSB') ,
#                                          'qend'].tolist()
    try: width = df_prodigal_hits.loc[df_prodigal_hits['GroupID'] == g, 'Qlen'].tolist()[0]
    except IndexError: continue
    idx = 6
    try:
        df_osa = df_osa.sort_values(by = 'Length', ascending = False)
        osa_start, osa_end = df_osa.iloc[0]['Start'], df_osa.iloc[0]['End']
        ax2, idx = Plot_Gene_Annotations(df_osa_gbff, ax2, width, idx, osa_start, osa_end, color = 'green')
    except KeyError:
        print("Not in OSA")
        
    try:
        df_osb = df_osb.sort_values(by = 'Length', ascending = False)
        osb_start, osb_end = df_osb.iloc[0]['Start'], df_osb.iloc[0]['End']
        ax2, idx = Plot_Gene_Annotations(df_osb_gbff, ax2, width, idx, osb_start, osb_end, color = 'orange')
    except KeyError:
        print("Not in OSB")
    
    ax2, idx = Plot_Gene_Annotations(df_prodigal_hits.loc[df_prodigal_hits['GroupID'] == g], 
                                     ax2, width, idx, 0, width, offset = 0)   
    ax2.set_yticks([])
    
    ylim = max(np.max(osa_gbl_counts), np.max(osb_gbl_counts))
    Make_Circle_Plots(ax0,osa_gbl_counts,'green', f"Syn. OSA\n#Samples :{len(osa_contig_count)}", ylim)
    Make_Circle_Plots(ax1,osb_gbl_counts,'orange', f"Syn. OSB\n#Samples :{len(osb_contig_count)}", ylim)
    fig.suptitle("\n"+g.replace("_"," "))
    
    fig.tight_layout()
    fig.savefig(out_dir+"Novel_Groups_11102024/"+g+".pdf")
    plt.close("all")
    

Group_555
Group_665
Group_835
Group_1035
Group_1160
Not in OSB
Group_1206
Group_1280
Group_1287
Group_1341
Group_1365
Group_1390
Group_1501
Group_1565
Group_1580
Group_1635
Not in OSB
Group_1673
Group_1674
Not in OSA
Group_1708
Not in OSA
Group_1740
Group_1775
Group_1829
Group_1842
Not in OSB
Group_1855
Not in OSA
Group_1862
Not in OSA
Group_1903
Group_1937
Not in OSA
Group_1947
Group_1949
Not in OSA
Group_2033
Not in OSB
Group_2061
Not in OSA
Group_2074
Group_2079
Not in OSA
Group_2080
Not in OSB
Group_2081
Not in OSA
Group_2086
Not in OSB
Group_2095
Group_2183
Not in OSA
Group_2200
Group_2235
Not in OSB
Group_2266
Not in OSB
Group_2270
Group_2276
Not in OSA
Group_2294
Not in OSA
Group_2299
Group_2323
Not in OSA
Group_2343
Group_2353
Group_2359
Group_2388
Group_2398
Not in OSA
Group_2401
Not in OSB
Group_2422
Not in OSB
Group_2433
Not in OSA
Group_2442
Group_2477
Group_2493
Group_2523
Not in OSB
Group_2548
Not in OSB
Group_2551
Group_2565
Group_2594
Not in OSA
Group_2596
Not in OSA
Gr

In [45]:
import resource 
from PyPDF2 import PdfMerger

soft, hard = resource.getrlimit(resource.RLIMIT_NOFILE)
resource.setrlimit(resource.RLIMIT_NOFILE, (hard, hard))

pdfs = list(filtered_grp['GroupID'].unique())
merger = PdfMerger()
for pdf in pdfs:
    if pdf.startswith('Group'):
        try: merger.append(out_dir+'Novel_Groups_11102024/'+pdf+'.pdf')
        except FileNotFoundError: print(pdf, 'not found')
        
merger.write(out_dir+'Gene_Plots.pdf')
merger.close()

Group_3465 not found
Group_3678 not found
Group_3714 not found
